In [ ]:
using Dates
include("src/srcs/StreamSource.jl");
include("src/srcs/StreamEvent.jl");
include("src/ops/Op.jl");
include("src/ops/OpNone.jl");
include("src/ops/OpReturn.jl");
include("src/ops/OpCombineLatest.jl");

### PeriodicSource

In [ ]:
include("src/srcs/PeriodicSource.jl");

periodic_source = PeriodicSource(
    id="test",
    next=OpReturn(),
    start_date=DateTime(2017, 1, 1),
    end_date=DateTime(2017, 1, 10),
    period=Day(1),
    current_date=DateTime(2017, 1, 1),
    inclusive_end=false)

for _ in 1:12
    println(next!(periodic_source))
end

### IterableSource

In [ ]:
include("src/srcs/IterableSource.jl");

values = collect(1:15)

iterable_source = IterableSource(
    id=:test2,
    next=OpReturn(),
    data=values,
    date_fn=x -> DateTime(2020, 1, 1 + x))

for _ in 1:length(values)+1
    println(next!(iterable_source))
end

### DataFrameSource

In [ ]:
include("src/srcs/DataFrameSource.jl");

df = DataFrame(
    datetime=DateTime.(["2017-01-01", "2017-01-02", "2017-01-03"]),
    value=[1, 2, 3],
    text=["a", "bc", "def"])

# DataFrameRows
df_source = DataFrameSource(
    id=1,
    next=OpReturn(),
    data=df,
    date_fn=x -> x[:datetime])

for _ in 1:nrow(df)+1
    println(next!(df_source))
end

println('-'^50)

# NamedTuples
df_source = DataFrameSource(
    id=2,
    next=OpReturn(),
    data=df,
    date_fn=x -> x[:datetime],
    as_named_tuple=true)

for _ in 1:nrow(df)+1
    println(next!(df_source))
end

### simulate_chronological_stream

In [ ]:
include("src/srcs/simulation.jl");
include("src/ops/OpPrint.jl");
include("src/ops/OpFunc.jl");
include("src/ops/OpLag.jl");
include("src/ops/OpPrint.jl");
include("src/pipeline.jl");

# periodic source
periodic_source = PeriodicSource(
    id=1,
    next=OpReturn(),
    start_date=DateTime(2017, 1, 1),
    end_date=DateTime(2017, 1, 12),
    period=Day(1),
    current_date=DateTime(2017, 1, 1))

# dates source
dates_source = IterableSource(
    id=2,
    next=OpReturn(),
    data=[DateTime(2017, 1, 2), DateTime(2017, 1, 5), DateTime(2017, 1, 10)],
    date_fn=x -> x)

# numbers source
numbers_source = IterableSource(
    id=3,
    next=OpReturn(),
    data=collect(1:9),
    date_fn=x -> DateTime(2017, 1, 1 + x))

simulate_chronological_stream((periodic_source, dates_source, numbers_source))

In [ ]:
using Statistics

include("src/ops/OpDropIf.jl");
include("src/ops/OpCombineLatest.jl");
include("src/aggs/AggPeriodFn.jl");
include("src/pipeline.jl");

combiner = @pipeline OpCombineLatest{StreamEvent}(;
    slot_map=Dict(
        :price => 1,
        :volume => 2,
        :volatility => 3
    ),
    key_fn=evt -> evt.source,
) OpDropIf(x -> any(isnothing.(x))) OpPrint()


# prices source
prices_source = IterableSource(
    id=1,
    data=[
        (DateTime(2020, 1, 1, 0, 0, 0), 101.3),
        (DateTime(2020, 1, 1, 0, 4, 0), 101.0),
        (DateTime(2020, 1, 1, 0, 6, 0), 99.05),
        (DateTime(2020, 1, 1, 0, 8, 0), 98.5),
        (DateTime(2020, 1, 1, 0, 10, 0), 101.2),
        (DateTime(2020, 1, 1, 0, 11, 0), 104.5),
        (DateTime(2020, 1, 1, 0, 12, 0), 105.0),
        (DateTime(2020, 1, 1, 0, 14, 0), 103.5),
        (DateTime(2020, 1, 1, 0, 15, 0), 105.6),
        (DateTime(2020, 1, 1, 0, 21, 0), 107.5)
    ],
    date_fn=x -> x[1],
    next=@pipeline AggPeriodFn{StreamEvent,DateTime}(;
        date_fn=evt -> evt.date,
        period_fn=date -> round_origin(date, Dates.Minute(5)),
        agg_fn=(period, buffer) -> begin
            StreamEvent(:price, period, last(map(x -> x.value[2], buffer)))
        end
    ) combiner
    # OpFunc(x -> println("price $x"), combiner)
)

# volumes source
volumes_source = IterableSource(
    id=2,
    data=[
        (DateTime(2020, 1, 1, 0, 0, 0), 1e6),
        (DateTime(2020, 1, 1, 0, 3, 0), 8e6),
        (DateTime(2020, 1, 1, 0, 4, 0), 12e7),
        (DateTime(2020, 1, 1, 0, 6, 0), 9e5),
        (DateTime(2020, 1, 1, 0, 7, 0), 6.5e6),
        (DateTime(2020, 1, 1, 0, 9, 0), 8.1e6),
        (DateTime(2020, 1, 1, 0, 11, 0), 9.8e6),
        (DateTime(2020, 1, 1, 0, 14, 0), 1.8e7),
        (DateTime(2020, 1, 1, 0, 16, 0), 1.2e7),
        (DateTime(2020, 1, 1, 0, 21, 0), 3e7)
    ],
    date_fn=x -> x[1],
    next=@pipeline AggPeriodFn{StreamEvent,DateTime}(;
        date_fn=evt -> evt.date,
        period_fn=date -> round_origin(date, Dates.Minute(5)),
        agg_fn=(period, buffer) -> begin
            StreamEvent(:volume, period, sum(x -> x.value[2], buffer))
        end
        ) combiner
        # OpFunc(x -> println("volume $x"))
)

# volatility source
volatility_source = IterableSource(
    id=3,
    data=[
        (DateTime(2020, 1, 1, 0, 0, 0), 0.0056),
        (DateTime(2020, 1, 1, 0, 3, 0), 0.0068),
        (DateTime(2020, 1, 1, 0, 6, 0), 0.0044),
        (DateTime(2020, 1, 1, 0, 9, 0), 0.008),
        (DateTime(2020, 1, 1, 0, 11, 0), 0.008)
    ],
    date_fn=x -> x[1],
    next=@pipeline AggPeriodFn{StreamEvent,DateTime}(;
        date_fn=evt -> evt.date,
        period_fn=date -> round_origin(date, Dates.Minute(5)),
        agg_fn=(period, buffer) -> begin
            StreamEvent(:volatility, period, mean(x -> x.value[2], buffer))
        end
    ) combiner
    # OpFunc(x -> println("volatility $x"), combiner);
)

sources = (prices_source, volumes_source, volatility_source)

simulate_chronological_stream(sources)